In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import lxml.html as lh
import re

from sqlalchemy import create_engine


In [2]:

engine = create_engine('sqlite://', echo=False)

In [12]:
states= pd.read_csv('states.csv')
# states=states.rename(columns={"State.1": "StateAbv"})\
#     .drop(columns=['State Capital','Region'])\
#     .reset_index(drop=True)
# Just to be sure no spaces remain in column for merge!!! 
states['State.1'] = states['State.1'].str.strip()
# states.convert_dtypes()
states.head()

,State,State.1,State Capital,Region
0,Alabama,AL,Montgomery,South
1,Alaska,AK,Juneau,West
2,Arizona,AZ,Phoenix,West
3,Arkansas,AR,Little Rock,South
4,California,CA,Sacramento,West


In [13]:
state =states["State.1"]
nat_park=[]  #This is where it all winds up

for name in state:
    st_park = [] # will be a list for list for all st parks- resets on each loop
    
    url=f'https://stateparks.com/{name}.html'
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    results= soup.find('div', id="overview")
    body = results.find_all('div', style="display:inline-block;")
    
    for row_num in range(len(body)): 
        row = [] # this will hold entries for one row- resets each loop
        for row_item in body[row_num]:
            aa = re.sub("(\xa0)|(\n)|,|•| "," ",row_item.text)
            if len(row) >1:
                del row[0]    
            row.append(aa)
        st_park.append(row)
  
    nat_park.append(st_park)

In [14]:
st_park

[['', 'National Memorial'], ['2', ' National Historic Sites']]

In [15]:
df=pd.DataFrame(nat_park)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,"[24, State Parks]","[3, State Forests]","[4, State Wildlife Management Areas]","[, National Park]","[4, National Forests]","[2, National Historic Sites]","[, National Historic Park]","[2, National Monuments]","[10, National Wildlife Refuges]",None,None,None,None,None,None,None,None,None,None
1,"[4, State Parks]","[2, National Parks]","[2, National Forests]","[2, National Historic Parks]","[2, National Monuments]","[22, National Wildlife Refuges]","[, National Recreation Trail]","[7, National Park and Preserves]",None,None,None,None,None,None,None,None,None,None,None
2,"[16, State Parks]","[, State Recreation Area]","[, State Historic Site]","[, State Fish Hatchery]","[, State Wildlife Area]","[4, National Parks]","[8, National Forests]","[, National Memorial]","[, National Historic Site]","[9, National Monuments]","[, National Wildlife Area]","[7, National Wildlife Refuges]","[, National Recreation Area]","[4, National Recreation Trails]","[, Metro and City Parks]",None,None,None,None
3,"[36, State Parks]","[, State Forest]","[, State Historic Site]","[2, State Fish Hatcherys]","[, State Natural Area]","[12, State Wildlife Management Areas]","[, National Park]","[3, National Forests]","[, National Memorial]","[3, National Historic Sites]","[9, National Wildlife Refuges]","[, National Recreation Trail]",None,None,None,None,None,None,None
4,"[109, State Parks]","[9, State Recreation Areas]","[6, State Forests]","[5, State Historic Sites]","[5, State Fish Hatcherys]","[, State Nature Preserve]","[10, State Reserves]","[10, State Wildlife Areas]","[9, National Parks]","[19, National Forests]","[, National Memorial]","[3, National Historic Sites]","[5, National Monuments]","[31, National Wildlife Refuges]","[3, National Recreation Areas]","[6, National Recreation Trails]","[, National Seashore]","[, National Park and Preserve]","[, Metro and City Parks]"
5,"[6, State Parks]","[2, State Forests]","[, State Fish Hatchery]","[5, State Wildlife Areas]","[3, National Parks]","[11, National Forests]","[, National Historic Landmark]","[, National Monument]","[4, National Wildlife Refuges]","[2, National Recreation Areas]",None,None,None,None,None,None,None,None,None
6,"[53, State Parks]","[25, State Forests]","[2, State Wildlife Areas]","[, Metro and City Parks]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,"[15, State Parks]","[2, State Forests]","[, State Wildlife Area]","[3, State Wildlife Management Areas]","[2, National Wildlife Refuges]",None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,"[114, State Parks]","[5, State Forests]","[, State Memorial]","[17, State Historic Sites]","[3, State Reserves]","[19, State Preserves]","[3, National Parks]","[3, National Forests]","[2, National Memorials]","[, National Historic Landmark]","[, National Monument]","[20, National Wildlife Refuges]","[, National Seashore]",None,None,None,None,None,None
9,"[53, State Parks]","[, State Forest]","[, State Memorial]","[15, State Historic Sites]","[, State Nature Preserve]","[, State Wildlife Management Area]","[, State Wildlife Refuge]","[2, National Forests]","[, National Historic Site]","[9, National Wildlife Refuges]","[, National Recreation Area]","[, National Seashore]","[2, Army Corps of Engineerss]",None,None,None,None,None,None


In [16]:
parks = pd.DataFrame(df[0].tolist(), columns=['Count', 'Type'])
parks.head()

,Count,Type
0,24,State Parks
1,4,State Parks
2,16,State Parks
3,36,State Parks
4,109,State Parks


In [17]:
stateparks= parks.join(states)
stateparks.head()

,Count,Type,State,State.1,State Capital,Region
0,24,State Parks,Alabama,AL,Montgomery,South
1,4,State Parks,Alaska,AK,Juneau,West
2,16,State Parks,Arizona,AZ,Phoenix,West
3,36,State Parks,Arkansas,AR,Little Rock,South
4,109,State Parks,California,CA,Sacramento,West


In [18]:
stateparks.to_csv("stateparks.csv")

In [19]:
stateparks.to_sql('stateparks', con=engine, if)

ValueError: Table 'stateparks' already exists.